In [2]:
import os
from dotenv import load_dotenv
import dtlpy as dl
from rfdetr.util.coco_classes import COCO_CLASSES



In [3]:
use_rc_env = False
if use_rc_env:
    dl.setenv('rc')
else:
    dl.setenv('prod')
if dl.token_expired():
    dl.login()


In [4]:
if use_rc_env:
    project = dl.projects.get(project_name='HusamDimo')
else:
    project = dl.projects.get(project_name='ShadiDemo')

In [12]:
models = project.models.list().print()



+----+--------------------------+----------------------------------------------------------+--------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [4]:
# Publish your app
dpk = project.dpks.publish()
# Install or update the application
try:
    app = project.apps.get(app_name=dpk.display_name)
    app.dpk_version = dpk.version
    app.update()
except dl.exceptions.NotFound:
    print("installing ...")
    app = project.apps.install(dpk=dpk)

Command Progress: 100%|██████████| 100/100 [00:02<00:00, 44.26it/s]


In [5]:
model = project.models.get(model_name = 'rf-detr')

In [27]:
model.open_in_web()

set the lables from COCO_CLASSES

In [6]:
model.labels = list(COCO_CLASSES.values())

In [14]:
len(COCO_CLASSES.values())


80

In [8]:
artifact = model.artifacts.upload(filepath='rf-detr-base-coco.pth')


Upload Items: 355MB [01:07, 5.50MB/s]                           
Iterate Entity: 100%|██████████| 1/1 [00:00<00:00,  6.67it/s]


upload the weights file


In [9]:
print(f"model status before artifact upload: {model.status}")
# Upload your weights
artifact = model.artifacts.upload(filepath='rf-detr-base-coco.pth')

# Update the configuration
model.configuration['weights_filename'] = artifact.filename


model = project.models.get(model_name = 'rf-detr')
print(f"model status after artifact upload: {model.status}")


model = project.models.get(model_name = 'rf-detr')
print(f"model status after deploy: {model.status}")

model status before artifact upload: deployed
Upload Items: 355MB [01:04, 5.81MB/s]                           
Iterate Entity: 100%|██████████| 1/1 [00:00<00:00,  6.19it/s]
model status after artifact upload: deployed
model status after deploy: deployed


In [30]:
model.status = 'trained'
model.update()

# # # Deploy your model
# print("-##################\n##################\n##################\n##################\n##################\n deploy only once \n##################\n##################\n##################\n##################\n##################\n")
# model.deploy()

-##################
##################
##################
##################
##################
 deploy only once 
##################
##################
##################
##################
##################



Service(created_at='2025-04-22T11:55:31.529Z', creator='husam.m@dataloop.ai', version='1.0.0', package_id='6807837fe3a6263781be39c0', package_revision='1.0.0', bot='bot.fcf28415-1e5f-4eb1-8cd7-47ec60297290@bot.dataloop.ai', init_input={'model_entity': '68078380f41fe36374d2c6ce'}, module_name='rf-detr-module', name='predict-68078380f41fe36374d2c6-0nca', url='https://gate.dataloop.ai/api/v1/services/680783b3db21547d3fa7decf', id='680783b3db21547d3fa7decf', active=True, queue_length_limit=None, run_execution_as_process=False, execution_timeout=3600, drain_time=600, on_reset='failed', _type=None, project_id='43e11685-1e6d-493b-a76d-41c33bbf0470', org_id='18739a63-4393-43ea-a87e-9a284b14978f', is_global=False, max_attempts=3, metadata={'ml': {'modelId': '68078380f41fe36374d2c6ce', 'modelOperation': 'deploy'}}, updated_by=None, app={'id': '6807838066f92076a27473f6', 'dpkName': 'rf-detr', 'dpkVersion': '1.0.0', 'dpkId': '6807837fe3a6263781be39c0'}, integrations=[])

In [8]:
cloned_model_taxi = project.models.get(model_name='rf-detr-tex4l')
artifact = cloned_model_taxi.artifacts.upload(filepath='model.pth')

# Update the configuration
cloned_model_taxi.configuration['weights_filename'] = artifact.filename

model.update()


NotFound: ('404', 'Model not found. Name: rf-detr-tex4l')

In [19]:
model = project.models.get(model_name = 'rf-detr')

dataset = project.datasets.get(dataset_name='husam_test_ds' if use_rc_env else 'husam_dogs_ds')
image_url = 'https://media.roboflow.com/notebooks/examples/dog-2.jpeg'
item_3 = dataset.items.upload(image_url)
image_url = 'https://media.roboflow.com/notebooks/examples/dog-3.jpeg'
item_4 = dataset.items.upload(image_url)
prediction_3 = model.predict(item_ids=[item_3.id,item_4.id],dataset_id=dataset.id)
prediction_3.wait()
item_3.open_in_web()

Upload Items: 100%|██████████| 1/1 [00:00<00:00,  2.25it/s]


In [7]:
item_3.open_in_web()

In [5]:
model = project.models.get(model_name = 'rf-detr')
model.open_in_web()
project.datasets.get(dataset_name='husam_train_dataset').open_in_web()

In [6]:

# Train with data splitting and validation
train_filters = dl.Filters(field='dir', values='/train')
val_filters = dl.Filters(field='dir', values='/validation')


model = model.clone(
    model_name='rd-dert-animals-sdk',
    description='Husam trained model',
    dataset=project.datasets.get(dataset_name='husam_train_dataset'),
    project_id=project.id,
    train_filter=train_filters,
    validation_filter=val_filters,
)

In [ ]:
exec = model.train()

In [ ]:
s = dl.services.get(service_id="67b4cb48e26f1d1d9699a4c4")
s.log(system=True,
          follow=True)